In [2]:
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Load UCI Adult Dataset
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
data = pd.read_csv(url, header=None, names=column_names, na_values=' ?', skipinitialspace=True)
data.dropna(inplace=True)

# Encode income
data['income'] = data['income'].apply(lambda x: 1 if x == '>50K' else 0)

# Encode categorical features
categorical_cols = data.select_dtypes(include='object').columns
label_encoders = {col: LabelEncoder().fit(data[col]) for col in categorical_cols}
for col, le in label_encoders.items():
    data[col] = le.transform(data[col])

# Sensitive attribute: sex (for fairness)
sensitive_feature = data['sex'].values

# Prepare features and labels
X = data.drop('income', axis=1)
y = data['income']

# Train/Test Split
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X, y, sensitive_feature, test_size=0.2, random_state=42)

# Scale Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define Task and Audit Networks
class TaskNet(nn.Module):
    def __init__(self, input_dim):
        super(TaskNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 2)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class AuditNet(nn.Module):
    def __init__(self, input_dim=2):
        super(AuditNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 1)
    def forward(self, x):
        return torch.sigmoid(self.fc2(F.relu(self.fc1(x))))

class SANNet(nn.Module):
    def __init__(self, input_dim):
        super(SANNet, self).__init__()
        self.task_net = TaskNet(input_dim)
        self.audit_net = AuditNet()
    def forward(self, x):
        task_out = self.task_net(x)
        audit_out = self.audit_net(task_out)
        return task_out, audit_out

# Model Training
model = SANNet(X_train.shape[1])
task_opt = optim.Adam(model.task_net.parameters(), lr=0.001)
audit_opt = optim.Adam(model.audit_net.parameters(), lr=0.001)
task_loss_fn = nn.CrossEntropyLoss()
audit_loss_fn = nn.BCELoss()

for epoch in range(10):
    model.train()
    task_out, audit_out = model(X_train_tensor)
    task_loss = task_loss_fn(task_out, y_train_tensor)
    audit_loss = audit_loss_fn(audit_out.squeeze(), y_train_tensor.float())
    total_loss = task_loss + audit_loss
    task_opt.zero_grad()
    audit_opt.zero_grad()
    total_loss.backward()
    task_opt.step()
    audit_opt.step()
    print(f"Epoch {epoch+1} - Loss: {total_loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    task_preds = model.task_net(X_test_tensor).argmax(dim=1)
    audit_scores = model.audit_net(model.task_net(X_test_tensor)).squeeze()
    audit_preds = (audit_scores > 0.5).float()

accuracy = accuracy_score(y_test_tensor, task_preds)
audit_accuracy = (audit_preds == y_test_tensor.float()).float().mean().item()

print(f"\n✅ TaskNet Accuracy: {accuracy:.4f}")
print(f"✅ AuditNet Ethical Compliance Accuracy: {audit_accuracy:.4f}")

# Fairness Metrics
def check_demographic_parity(preds, sensitive):
    for group in np.unique(sensitive):
        rate = np.mean(preds[sensitive == group] == 1)
        print(f"Demographic Parity (Group {group}): {rate:.4f}")

def check_equalized_odds(preds, labels, sensitive):
    for group in np.unique(sensitive):
        group_preds = preds[sensitive == group]
        group_labels = labels[sensitive == group]
        cm = confusion_matrix(group_labels, group_preds)
        if cm.shape == (2, 2):
            TN, FP, FN, TP = cm.ravel()
            FPR = FP / (FP + TN) if (FP + TN) else 0
            TPR = TP / (TP + FN) if (TP + FN) else 0
            print(f"Group {group} - FPR: {FPR:.4f}, TPR: {TPR:.4f}")
        else:
            print(f"Group {group} - Not enough data for confusion matrix.")

check_demographic_parity(task_preds.numpy(), sensitive_test)
check_equalized_odds(task_preds.numpy(), y_test_tensor.numpy(), sensitive_test)


Epoch 1 - Loss: 1.2833
Epoch 2 - Loss: 1.2718
Epoch 3 - Loss: 1.2607
Epoch 4 - Loss: 1.2500
Epoch 5 - Loss: 1.2397
Epoch 6 - Loss: 1.2297
Epoch 7 - Loss: 1.2201
Epoch 8 - Loss: 1.2108
Epoch 9 - Loss: 1.2019
Epoch 10 - Loss: 1.1932

✅ TaskNet Accuracy: 0.7827
✅ AuditNet Ethical Compliance Accuracy: 0.7588
Demographic Parity (Group 0): 0.0428
Demographic Parity (Group 1): 0.1110
Group 0 - FPR: 0.0354, TPR: 0.1030
Group 1 - FPR: 0.0472, TPR: 0.2564
